In [73]:
import json
import paho.mqtt.client as mqtt

In [74]:
host = "www.ds.se.shibaura-it.ac.jp"
topic = "/saito/training/201802/shelter/sensor/sh47517777"

In [75]:
send_data = {"tmp":40, "hum":40, "lux":700, "co2":456}
send_message = json.dumps(send_data)

In [76]:
send_message

'{"tmp": 40, "hum": 40, "lux": 700, "co2": 456}'

In [77]:
client = mqtt.Client(protocol=mqtt.MQTTv311)
client.connect(host, port=1883, keepalive=60)
client.publish(topic, send_message)
client.disconnect()

0

In [8]:
hoge = {'a':123, 'b':456}

In [11]:
for k, v in hoge.items():
    print(k, v)

a 123
b 456


In [7]:
hoge

{'a': 789}

In [14]:
?next()

In [49]:
import json
import time
import datetime
import threading
import paho.mqtt.client as mqtt

In [50]:
host= "www.ds.se.shibaura-it.ac.jp"
publish_topic_base =    "/saito/training/201802/shelter/app/"
subscribe_topic_base =  "/saito/training/201802/shelter/support/"
port = 1883
threads = {}

In [3]:
class AgentBase():
    def __init__(self, agent_ID, agent_STATE):
        self.agent_ID = agent_ID
        self.agent_state = agent_STATE
        self.thread_flag = False
    def publish(self, pub_host, pub_topic):
        publish_MQTT_client = mqtt.Client(client_id="publish_"+str(self.agent_ID))
        publish_MQTT_client.connect(host=pub_host, port=1883,keepalive=60)
        message = json.dumps(self.agent_state)
        publish_MQTT_client.publish(topic=pub_topic, payload=message)
        publish_MQTT_client.disconnect()
    def subscribe(self, sub_host, sub_topic):

        def on_connect(client, userdata, flags, respons_code):
            client.subscribe(sub_topic)
                
        def on_message(client, userdata, msg):
            new_agent_state = json.loads(msg.payload.decode())
            for key, value in new_agent_state.items():
                self.agent_state[key] = value
            print("エージェントID:{}の状態が変化しました".format(self.agent_ID))

        subscribe_MQTT_client = mqtt.Client(client_id="subscribe_"+str(self.agent_ID))
        subscribe_MQTT_client.on_connect = on_connect
        subscribe_MQTT_client.on_message = on_message
        
        subscribe_MQTT_client.connect(host=sub_host, port=1883, keepalive=60)
        while not self.thread_flag:
            subscribe_MQTT_client.loop()

In [5]:
test_agent_base_class = AgentBase(1,{'state':'test' })

In [6]:
test_agent_base_class.publish(host, pub_topic='/saito/test')

In [7]:
thread = threading.Thread(target=test_agent_base_class.subscribe, args=(host, '/saito/test'))

In [15]:
print(test_agent_base_class.agent_state)
print(test_agent_base_class.thread_flags)

{'state': 'test', 'new_state2': 'hugahuga'}
True


In [8]:
thread.start()

In [52]:
MQTT_client = mqtt.Client()
MQTT_client.connect(host)
MQTT_client.publish('/saito/training/201802/shelter/support/sh30406024',json.dumps({'new_state3':'aaaaaa'}))
MQTT_client.disconnect()

0

In [13]:
thread.is_alive()

False

In [12]:
test_agent_base_class.thread_flags = True

In [4]:
class Agent(AgentBase):
    def __init__(self, agent_id, agent_state):
        super().__init__(agent_ID=agent_id, agent_STATE=agent_state)
        publish_topic = publish_topic_base + self.agent_state['code']
        subscribe_topic = subscribe_topic_base + self.agent_state['code']
        super().publish(pub_host=host, pub_topic=publish_topic)
        self.temp_thread = threading.Thread(target=super().subscribe, args=(host, subscribe_topic))
        self.temp_thread.start()
        
    def step(self, *args, **kwargs):
        print("Step処理実行中")
        pass
    
    def terminate(self):
        print("終了処理")
        self.thread_flag = True

In [11]:
test_agent1 = Agent(1, {'code':'sh12345678'})
test_agent2 = Agent(2, {'code':'sh87654321'})

In [17]:
print(test_agent1.agent_state)
print(test_agent2.agent_state)

{'code': 'sh12345678'}
{'code': 'sh87654321', 'new_state3': 'aaaaaa'}


In [10]:
test_agent1.temp_thread.isAlive()

False

In [8]:
test_agent1.step()

Step処理実行中


In [15]:
test_agent1.terminate()
test_agent2.terminate()

終了処理
終了処理


In [24]:
class AgentSet():
    def __init__(self):
        self.agents = []
    def define_agent(self, agent_id, agent_state):
        class _Agent(AgentBase):
            def __init__(self, agent_id, agent_state):
                super().__init__(agent_ID=agent_id, agent_STATE=agent_state)
                publish_topic = publish_topic_base + self.agent_state['code']
                subscribe_topic = subscribe_topic_base + self.agent_state['code']
                super().publish(pub_host=host, pub_topic=publish_topic)
                self.temp_thread = threading.Thread(target=super().subscribe, args=(host, subscribe_topic))
                self.temp_thread.start()

            def step(self, *args, **kwargs):
                print("Step処理実行")
                pass

            def terminate(self):
                print("終了処理")
                self.thread_flag = True
                
        return _Agent(agent_id, agent_state)
    
    def generate_agent(self, n):
        for i in range(n):
            temp_agent = self.define_agent(i, {'code':'sh'+str(10000000+i)})
            self.agents.append(temp_agent)
            
    def step(self):
        for agent in self.agents:
            agent.step()
            
    def terminate_agent(self):
        for agent in self.agents:
            agent.terminate()

In [25]:
test_agent_set = AgentSet()

In [26]:
test_agent_set.generateAgent(3)

{'code': 'sh10000002', 'new_state3': 'aaaaaa'}

In [1]:
import csv
import pandas as pd

In [39]:
df = pd.read_csv('shelter_data.csv')
df.to_csv('hoge.csv')

In [45]:
f = open('hoge.csv','r')
reader = csv.reader(f)

In [46]:
header = next(reader)

In [47]:
header

['番号',
 '施設名',
 'lat',
 'lon',
 '所在地',
 '収容人数',
 '連絡先',
 '避難者数',
 '内訳男性',
 '内訳女性',
 '飲料水',
 '食事',
 '使用可能トイレ',
 '電気',
 'ガス',
 '生活用水',
 '固定電話',
 '携帯電話',
 'データ通信',
 '救護所設置',
 '医療チームの巡回',
 '昼間人数',
 '夜間人数',
 '75歳以上',
 '未就学児',
 '過密度',
 '毛布等寝具',
 '室温度管理',
 '手洗い環境',
 'トイレ掃除',
 '土足禁止',
 '下水',
 'ごみ集積場所',
 '館内禁煙',
 'ペット収容所',
 '下痢',
 '嘔吐',
 '咳・痰',
 '発熱',
 '人工呼吸器',
 '在宅酸素',
 '透析',
 '糖尿病（要インスリン）',
 '緊急性のある精神疾患',
 '緊急歯科要治療',
 '緊急処置を要する妊婦',
 '要配慮者',
 'うち医療的配慮者',
 'うち福祉的配慮者',
 'うち外国人',
 'インフルエンザ',
 '感染性胃腸炎',
 '食中毒',
 'その他緊急事項']